In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime

ModuleNotFoundError: No module named 'pandas'

In [27]:
df_kulutus = pd.read_csv("../Data/Kulutusennuste_Tammikuu_2021.csv")
df_tuotanto = pd.read_csv("../Data/Sahkontuotantoennuste_Tammikuu_2021.csv")

In [65]:
def ChargingTime(interval, time, arr):
    # Käydään läpi ennustearvot ja lasketaan 'time' pituinen liukuva keskiarvo
    # Keskiarvot taulukkoon jossa sarakkeet 'aloitusaika', 'lopetusaika' ja 'keskiarvo'
    # Keskiarvoista valitaan pienin
    # Return pienimmän keskiarvon aloitusaika
    avgs = []

    # Calculates averages
    for i in range(len(arr) - time + 1):
        t1 = arr[i][0]
        t2 = arr[i][1]

        if (pd.Timedelta(t2-t1).seconds / 60.0) > interval:
            break
        else:
            window = arr[i : i + time]
            
            sum = 0
            for j in window:
                sum = sum + j[2]

            window_avg = round(sum / time, 3)
            
            if (len(arr) - 1) > i + time:
                avgs.append([arr[i], arr[i + time], window_avg])
    
    # Find min average and select the starting time to achieve that
    min_avg = float('inf')
    for k in avgs:
        if min_avg > k[2]:
            min_avg = k[2]

    return min_avg

In [18]:
len(df_kulutus)

8928

In [40]:
# Muutetaan arvot datetime:ksi
df_kulutus["Alkuaika UTC+02:00"] = pd.to_datetime(df_kulutus["Alkuaika UTC+02:00"])
df_kulutus["Lopetusaika UTC+02:00"] = pd.to_datetime(df_kulutus["Lopetusaika UTC+02:00"])

# Nimetään aikaväli
start_date = pd.to_datetime("2020-12-31 22:00:00")
end_date = pd.to_datetime("2021-01-01 05:00:00")

# Rajataan aikavälin mukaan
mask = (df_kulutus['Alkuaika UTC+02:00'] > start_date) & (df_kulutus['Lopetusaika UTC+02:00'] <= end_date)
df_kulutus_filt = df_kulutus.loc[mask]

# Valitaan vain kolme saraketta
df_kulutus_filt = df_kulutus_filt[['Alkuaika UTC+02:00', 'Lopetusaika UTC+02:00', 'Kulutusennuste - seuraava vuorokausi']]
df_kulutus_filt.head()

,Alkuaika UTC+02:00,Lopetusaika UTC+02:00,Kulutusennuste - seuraava vuorokausi
0,2021-01-01 00:00:00,2021-01-01 00:05:00,9743.621
1,2021-01-01 00:05:00,2021-01-01 00:10:00,9743.621
2,2021-01-01 00:10:00,2021-01-01 00:15:00,9743.621
3,2021-01-01 00:15:00,2021-01-01 00:20:00,9743.621
4,2021-01-01 00:20:00,2021-01-01 00:25:00,9743.621


In [ ]:
x = df_kulutus_filt.values.tolist()
x

In [66]:
ChargingTime(120, 5, x)

8901.148

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

df_kulutus.head()
plt.plot(df_kulutus["Alkuaika UTC+02:00"], df_kulutus["Kulutusennuste - seuraava vuorokausi"])
plt.show()